In [2]:
import requests
from tqdm.notebook import tqdm
import pyspark
from pyspark.sql import SparkSession

In [3]:
spark = SparkSession.builder.appName("ProfParser3000").getOrCreate()

23/09/25 15:15:51 WARN Utils: Your hostname, g8rb8-desktop resolves to a loopback address: 127.0.1.1; using 192.168.0.48 instead (on interface enp8s0)
23/09/25 15:15:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/09/25 15:15:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/09/25 15:15:53 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
requests.get('https://www.cs.utexas.edu/people').content

b'<!DOCTYPE html>\n<!--[if IEMobile 7]><html class="iem7"  lang="en" dir="ltr"><![endif]-->\n<!--[if lte IE 6]><html class="lt-ie9 lt-ie8 lt-ie7"  lang="en" dir="ltr"><![endif]-->\n<!--[if (IE 7)&(!IEMobile)]><html class="lt-ie9 lt-ie8"  lang="en" dir="ltr"><![endif]-->\n<!--[if IE 8]><html class="lt-ie9"  lang="en" dir="ltr"><![endif]-->\n<!--[if (gte IE 9)|(gt IEMobile 7)]><!--><html  lang="en" dir="ltr" prefix="content: http://purl.org/rss/1.0/modules/content/ dc: http://purl.org/dc/terms/ foaf: http://xmlns.com/foaf/0.1/ og: http://ogp.me/ns# rdfs: http://www.w3.org/2000/01/rdf-schema# sioc: http://rdfs.org/sioc/ns# sioct: http://rdfs.org/sioc/types# skos: http://www.w3.org/2004/02/skos/core# xsd: http://www.w3.org/2001/XMLSchema#"><!--<![endif]-->\n\n<head>\n  <meta charset="utf-8" />\n<link rel="shortcut icon" href="https://www.cs.utexas.edu/sites/default/files/favicon.ico" type="image/vnd.microsoft.icon" />\n<meta name="description" content="Our faculty are at the heart of what 

In [5]:
sources = '>'.join([e for e in str(requests.get('https://www.cs.utexas.edu/people').content).split('>') if 'href' in e])

In [6]:
sources

'\\n<link rel="shortcut icon" href="https://www.cs.utexas.edu/sites/default/files/favicon.ico" type="image/vnd.microsoft.icon" />\\n<link rel="canonical" href="https://www.cs.utexas.edu/people" />\\n<link rel="shortlink" href="https://www.cs.utexas.edu/node/72685" />\\n\\n  <link type="text/css" rel="stylesheet" href="https://www.cs.utexas.edu/sites/default/files/css/css_lQaZfjVpwP_oGNqdtWCSpJT1EMqXdMiU84ekLLxQnc4.css" media="all" />\\n<link type="text/css" rel="stylesheet" href="https://www.cs.utexas.edu/sites/default/files/css/css_EaGgnBDR29Z3ZvyJqu3g05BwMAQ2tR8VX5jOTSDETAE.css" media="all" />\\n<link type="text/css" rel="stylesheet" href="https://www.cs.utexas.edu/sites/default/files/css/css_BwG27io1gAnLEo-qBJ8NhWd4Rwhkv5B_MR6Korfr09A.css" media="all" />\\n<link type="text/css" rel="stylesheet" href="https://www.cs.utexas.edu/sites/default/files/css/css_m8iJuN2Mk8QiiHHW5lpWRxGad6tQGvBuRS4F_PBf7ho.css" media="all" />\\n      <a href="#main-menu" class="element-invisible element-focus

In [7]:
prof_sites = list(set([e for e in sources.split('"') if 'faculty-researchers' in e]))

In [8]:
len(prof_sites)

137

In [9]:
def check_site(fp, keywords):
    full_fp = 'https://www.cs.utexas.edu' + fp
    content = requests.get(full_fp).text
    content = (content
               .replace('financial-information','')
               .replace('operating-systems-distributed-systems-and-networking','')
               .replace('security-and-privacy','')
              )
    rez = False
    for e in keywords:
        if e in content:
            rez = True
            break
    return rez

In [10]:
def find_keywords_old(keywords = ['distributed']):
    matches = []
    for fp in tqdm(prof_sites):
        if check_site(fp, keywords):
            matches.append(fp.split('/')[-1].replace('-',' '))
    return matches

In [11]:
def find_keywords(prof_sites, keywords = ['distributed']):
    matches = []
    profs = spark.sparkContext.parallelize(prof_sites)
    return (profs
            .filter(lambda x: check_site(x, keywords))
            .map(lambda x: x.split('/')[-1].replace('-',' '))
            .collect())

In [131]:
find_keywords(prof_sites)

['katrin erk',
 'e allen emerson',
 'simon peter',
 'robert van de geijn',
 'david zuckerman',
 'qiang liu',
 'aloysius k mok',
 'jayadev misra',
 'ken mcmillan',
 'vijay k garg',
 'ahmed gheith',
 'vijaya ramachandran']

In [132]:
find_keywords(prof_sites, ['assembly'])

[]

In [133]:
find_keywords(prof_sites, ['security'])

['emmett witchel',
 'alan bovik',
 'brent waters',
 'david zuckerman',
 'bruce porter',
 'swarat chaudhuri',
 'shravan narayan',
 'david wu',
 'david zhigang pan',
 'i%C5%9F%C4%B1l dillig',
 'simon lam',
 'calvin lin',
 'william young',
 'andrew hintz',
 'brian meixell',
 'scott aaronson',
 'hovav shacham',
 'bill press']

In [134]:
find_keywords(prof_sites, ['deep learning'])

['philipp kr%C3%A4henb%C3%BChl', 'qixing huang']

In [138]:
find_keywords(prof_sites, ['computer vision'])

['kristen grauman',
 'qixing huang',
 'garrett warnell',
 'roberto martin martin',
 'yuke zhu']

In [139]:
find_keywords(prof_sites, ['penetration testing'])

['andrew hintz']

In [141]:
find_keywords(prof_sites, ['linux'])

[]

In [142]:
find_keywords(prof_sites, ['operating systems'])

['aditya akella', 'supreeth shastri', 'ahmed gheith']

In [143]:
find_keywords(prof_sites, ['unix'])

[]

In [153]:
find_keywords(prof_sites, ['Netflix','Square','Airbnb','Google','Amazon','Spotify'])

['kevin tian',
 'inderjit dhillon',
 'eunsol choi',
 'atlas wang',
 'shuchi chawla',
 'swarat chaudhuri',
 'shravan narayan',
 'lili qiu',
 'ken mcmillan',
 'andrew hintz',
 'roberto martin martin',
 'alexandros dimakis',
 'lorenzo alvisi',
 'james bornholt',
 'joydeep biswas']

In [156]:
find_keywords(prof_sites, ['Morgan Stanley','Goldman','Morgan'])

['joydeep biswas']

In [12]:
find_keywords(prof_sites, ['cloud','AWS','GCP'])

['daehyeok kim',
 'dixin tang',
 'supreeth shastri',
 'aloysius k mok',
 'devdatta kulkarni']

In [13]:
find_keywords(prof_sites, ['GPU'])

['lizy k john',
 'emmett witchel',
 'christopher rossbach',
 'david wu',
 'stephen keckler',
 'calvin lin',
 'aditya akella',
 'david zhigang pan']

In [18]:
find_keywords(prof_sites, ['parallelism'])

['keshav pingali']